In [1]:
import mne
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.cuda.is_available()

True

In [7]:
CHB_files1 = {
    'CHB-MIT/CHB_Database/chb01/chb01_01.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_02.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_03.edf': [(0, 2396, 'interictal'), (2396, 2996, 'preictal'), (2996, 3036, 'ictal'), (3036, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_04.edf': [(0, 867, 'interictal'), (867, 1467, 'preictal'), (1467, 1494, 'ictal'), (1494, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_05.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_06.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_07.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_08.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_09.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_10.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_11.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_12.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_13.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_14.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_15.edf': [(0, 1132, 'interictal'), (1132, 1732, 'preictal'), (1732, 1772, 'ictal'), (1772, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_16.edf': [(0, 415, 'interictal'), (415, 1015, 'preictal'), (1015, 1066, 'ictal'), (1066, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_17.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_18.edf': [(0, 1120, 'interictal'), (1120, 1720, 'preictal'), (1720, 1810, 'ictal'), (1810, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_19.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_20.edf': [(0, 2663, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_21.edf': [(0, 327, 'preictal'), (327, 420, 'ictal'), (420, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_22.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_23.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_24.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_25.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_26.edf': [(0, 1262, 'interictal'), (1262, 1862, 'preictal'), (1862, 1963, 'ictal'), (1963, 2325, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_27.edf': [(0, 600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_29.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_30.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_31.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_32.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_33.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_34.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_36.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_37.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_38.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_39.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_40.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_41.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_42.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_43.edf': [(0, 3600, 'interictal')],
    'CHB-MIT/CHB_Database/chb01/chb01_46.edf': [(0, 3600, 'interictal')]
    }

In [3]:
#creating dataset
class CHBData(Dataset):
    def __init__(self, CHB_files, segment_length):
        self.segments = [] # input for model containing segment of length "l" and torch tensor containing the eeg values for that segment
        self.labels = [] # output of model contrainig  the labels of each segment, either interictal, preictal, or ictal
        for file_path, info in CHB_files.items():
            processed_data=self.preprocessing(file_path) # preprocess every file in dictionary 
            for start, end, label in info:
                segmented_eeg = self.segment_eeg(processed_data,start, end, label,segment_length) # segments that file
                for segment,label in segmented_eeg:
                    self.segments.append(segment) # adds to final list for model 
                    self.labels.append(label)

    def __len__(self):
        return len(self.segments)
    
    def __getitem__(self, i):
        return self.segments[i],self.labels[i]

    def segment_eeg(self,segment_tensor,start, end, label,segment_length):
        start=start*256 #have to mulyiply time by sampling rate BUDDDDYYYY 
        end=end*256
        segments=[] # list to store tuple of each segments pytorch tensor and each label inside the data 
        for i in range(start,end,segment_length):
            segment_end=min(i+segment_length,end)
            segment=segment_tensor[:,i:segment_end] # isolates segment of wtv lenght it is from each torch tensor in preprocessing our data 
            segments.append((segment,label))
            
        return segments

    
    def preprocessing(self,file_path):
        # loading data: 
        raw = mne.io.read_raw_edf(file_path)
        raw.load_data()
        # processing every raw object to remove 60 hz and its multiples:
        eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
        freqs = (60,120)
        raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
        # applying a high pass filter of order 4 with a cutoff frequency of 30 Hz to the data to enhance gamma signal to noise ratio:
        raw_notch.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
        numpy_array=raw_notch.get_data()
        segment_tensor=torch.from_numpy(numpy_array)
        return segment_tensor

In [8]:
dataset = CHBData(CHB_files1, 512)

Extracting EDF parameters from c:\Users\matth\PythonProjects\SeizureSense\Notebooks for testing\CHB-MIT\CHB_Database\chb01\chb01_01.edf...
EDF file detected


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\matth\\PythonProjects\\SeizureSense\\Notebooks for testing\\CHB-MIT\\CHB_Database\\chb01\\chb01_01.edf'

In [1]:
class CONV(nn.Module):
    def __init__(self):
        super(CONV, self).__init__()
        # Goes over each channel horizontally (temporal)
        self.conv1 = nn.Conv2d(1, 16, (1, 128))

        self.batchnorm1 = nn.BatchNorm2d(16)
        # Goes over all 23 channels at the same time to find spatial patterns in the new temporal filters 
        self.conv2 = nn.Conv2d(16, 32, (23,1))

        self.batchnorm2 = nn.BatchNorm2d(32)
        # Squishes all of the channels down into a 1x(96 or smaller) output to extract the most important parts of the data
        self.pooling2 = nn.MaxPool2d((1,23),(1,2))

        self.conv3 = nn.Conv1d(32, 64, 1)

        self.batchnorm3 = nn.BatchNorm1d(64)

        self.pooling3 = nn.MaxPool1d(2, 2)

        self.f1 = nn.Linear(2612, 3)

        self.dropout = nn.Dropout(0.3)
    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Layer 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.pooling2(x)
        x = self.dropout(x)

        # Layer 3
        x = x.view(1,32,182)
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = self.pooling3(x)
        
        # Fully connected
        x = x.view(5824)
        x = F.sigmoid(self.f1(x))
        return x

NameError: name 'nn' is not defined

tensor([0.4455], grad_fn=<SigmoidBackward0>)
